Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [1]:
NAME = "Lauren Murphy"
COLLABORATORS = "Kaitlyn Downey, Riley Poduje"

---

## Homework 1 - Tidy Data and Fielding Independent Pitching

* Note: Feel free to use either Excel or Python for this assignment.  Please submit Python via JupyterHub and Excel via Blackboard.

Dataset: 

hw_df_pivot = pd.read_csv("https://raw.githubusercontent.com/TravisRPetersen/sports-analytics-spring18/master/homework_01/pitcher_data_untidy.csv")

---
## Part 1: Tidy & Clean Data

* Use the pandas melt and pivot functions tidy the dataset
* Check for data integrity issues
* Use the loc method to replace any erroneous or missing data

In [2]:
import pandas as pd
import numpy as np 
hw_df_pivot=pd.read_csv("https://raw.githubusercontent.com/TravisRPetersen/sports-analytics-spring18/master/homework_01/pitcher_data_untidy.csv")

In [3]:
hw_df_pivot.head(1)

,nameFull,HR_2012,HR_2013,HR_2014,HR_2015,BB_2012,BB_2013,BB_2014,BB_2015,HBP_2012,...,HBP_2014,HBP_2015,SO_2012,SO_2013,SO_2014,SO_2015,IPouts_2012,IPouts_2013,IPouts_2014,IPouts_2015
0,A. J. Burnett,18.0,11.0,20.0,11.0,62.0,67.0,96.0,49.0,9.0,...,16.0,11.0,180.0,209.0,190.0,143.0,607.0,573.0,641.0,492.0


In [4]:
melt_hw_df=pd.melt(hw_df_pivot,id_vars='nameFull')

In [5]:
melt_hw_df.head()

,nameFull,variable,value
0,A. J. Burnett,HR_2012,18.0
1,Chris Sale,HR_2012,19.0
2,Clayton Kershaw,HR_2012,16.0
3,Cole Hamels,HR_2012,24.0
4,Dan Haren,HR_2012,28.0


In [6]:
melt_hw_df['nameFull'].value_counts()

R. A. Dickey         20
Edinson Volquez      20
Jon Lester           20
A. J. Burnett        20
Cole Hamels          20
Felix Hernandez      20
Lance Lynn           20
Madison Bumgarner    20
Ian Kennedy          20
Mike Leake           20
Clayton Kershaw      20
Rick Porcello        20
Zack Greinke         20
Jordan Zimmerman     20
Max Sherzer          20
Jeff Samardzija      20
David Price          20
Dan Haren            20
James Shields        20
Wade Miley           20
Jordan Zimmermann    20
Jeff Samarja         20
Mark Buehrle         20
Chris Sale           20
Yovani Gallardo      20
Name: nameFull, dtype: int64

In [7]:
melt_hw_df.loc[melt_hw_df['nameFull']=='Jordan Zimmermann','nameFull']='Jordan Zimmerman'

In [8]:
melt_hw_df['column']=melt_hw_df['variable'].apply(lambda x: x.split('_')[0])
melt_hw_df['yearID']=melt_hw_df['variable'].apply(lambda x: x.split('_')[1])
melt_hw_df.head()

,nameFull,variable,value,column,yearID
0,A. J. Burnett,HR_2012,18.0,HR,2012
1,Chris Sale,HR_2012,19.0,HR,2012
2,Clayton Kershaw,HR_2012,16.0,HR,2012
3,Cole Hamels,HR_2012,24.0,HR,2012
4,Dan Haren,HR_2012,28.0,HR,2012


In [9]:
pivot_hw_df=pd.pivot_table(melt_hw_df,index=['nameFull','yearID'],columns='column',values='value').reset_index()
pivot_hw_df.head()

column,nameFull,yearID,BB,HBP,HR,IPouts,SO
0,A. J. Burnett,2012,62.0,9.0,18.0,607.0,180.0
1,A. J. Burnett,2013,67.0,9.0,11.0,573.0,209.0
2,A. J. Burnett,2014,96.0,16.0,20.0,641.0,190.0
3,A. J. Burnett,2015,49.0,11.0,11.0,492.0,143.0
4,Chris Sale,2012,51.0,6.0,19.0,576.0,192.0


In [10]:
pivot_hw_df.describe().round(3)

column,BB,HBP,HR,IPouts,SO
count,92.000,92.000,91.000,92.000,92.000
mean,54.120,6.837,19.385,608.402,186.293
std,15.008,3.515,7.339,54.968,53.094
min,27.000,0.000,-28.000,492.000,91.000
25%,42.750,4.000,16.000,575.250,147.750
50%,51.500,6.500,19.000,607.500,181.500
75%,64.500,9.000,23.500,643.000,214.250
max,105.000,16.000,35.000,745.000,500.000


In [11]:
pivot_hw_df[pivot_hw_df['HR']==-28]

column,nameFull,yearID,BB,HBP,HR,IPouts,SO
68,Mike Leake,2012,41.0,3.0,-28.0,537.0,116.0


In [12]:
pivot_hw_df.loc[(pivot_hw_df['nameFull']=='Mike Leake')&
           (pivot_hw_df['yearID']=='2012'),'HR'] = 2.0

In [13]:
pivot_hw_df.isnull().sum()

column
nameFull    0
yearID      0
BB          0
HBP         0
HR          1
IPouts      0
SO          0
dtype: int64

In [14]:
pivot_hw_df[pivot_hw_df['HR'].isnull()]

column,nameFull,yearID,BB,HBP,HR,IPouts,SO
32,Ian Kennedy,2012,55.0,14.0,NaN,625.0,187.0


In [15]:
pivot_hw_df.loc[(pivot_hw_df['nameFull']=='Ian Kennedy')&
           (pivot_hw_df['yearID']=='2012'), 'HR'] = 0

---
## Part 2: Calculate Fielding Independent Pitching

Calculate FIP for each pitcher/year combination.  The FIP constant table is below.

FIP Constant:

2012: 3.095

2013: 3.048

2014: 3.132

2015: 3.134

In [16]:
pivot_hw_df['FIPC']=np.where(pivot_hw_df['yearID']=='2012', 3.095, (np.where(pivot_hw_df['yearID']=='2013',  3.048, (np.where(pivot_hw_df['yearID']=='2014', 3.132, 3.134)))))

In [17]:
def calc_fip(row):
    hr=13*row['HR']
    bbhbp=3*(row['BB']+row['HBP'])
    k=2*row['SO']
    ip=row['IPouts']
    fipc=row['FIPC']
    fip=((hr+bbhbp-k)/ip)+fipc
    return fip

In [18]:
pivot_hw_df['FIP'] = pivot_hw_df.apply(lambda row: calc_fip(row), axis=1)

---
## Part 3: Lowest FIP

* Produce a DataFrame with the lowest 5 single-season FIPs of our dataset

In [19]:
(pivot_hw_df
 .sort_values('FIP', ascending=True)
 .head()
 .round(3)
 .reset_index(drop=True)
[['nameFull','yearID','FIP']])

column,nameFull,yearID,FIP
0,Felix Hernandez,2014,2.229
1,Clayton Kershaw,2014,2.692
2,Clayton Kershaw,2015,2.753
3,Clayton Kershaw,2013,2.828
4,Ian Kennedy,2012,2.828
